In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.output_parsers.datetime import DatetimeOutputParser
from langchain_classic.output_parsers.boolean import BooleanOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_classic.output_parsers import OutputFixingParser
from dotenv import load_dotenv

load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

## Output Parsers

**String Parser**

In [ ]:
llm.invoke("hello")

In [ ]:
llm.invoke("hello").content

In [ ]:
parser = StrOutputParser()

In [ ]:
parser.invoke(
    llm.invoke("hello")
)

### Other Parsers

**Datetime**

In [ ]:
llm.invoke(
    "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
    "Don't say anything else"
)

In [ ]:
parser = DatetimeOutputParser()

In [ ]:
parser.invoke(
    llm.invoke(
        "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
        "Don't say anything else"
    )
)

**Boolean**

In [ ]:
llm.invoke(
    "Are you an AI? YES or NO only"
)

In [ ]:
parser = BooleanOutputParser()

In [ ]:
parser.invoke(
    input=llm.invoke(
        "Are you an AI? YES or NO only"
    )
)

In [ ]:
parser.invoke(
    input=llm.invoke(
        "Are you Human? YES or NO only"
    )
)

## Structured

**Dict Schema**

In [ ]:
from typing_extensions import Annotated, TypedDict

class UserInfo(TypedDict):
    """User's info."""
    name: Annotated[str, "", "User's name. Defaults to ''"]
    country: Annotated[str, "", "Where the user lives. Defaults to ''"]


In [ ]:
llm_with_structure = llm.with_structured_output(UserInfo)

In [ ]:
llm_with_structure.invoke(
    "My name is Henrique, and I am from Brazil"
)

In [ ]:
llm_with_structure.invoke(
    "The sky is blue"
)

In [ ]:
llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)

**Pydantic**

In [ ]:
from pydantic import BaseModel, Field

class PydanticUserInfo(BaseModel):
    """User's info."""
    name: Annotated[str, Field(description="User's name. Defaults to ''", default=None)]
    country: Annotated[str, Field(description="Where the user lives. Defaults to ''", default=None, )]

In [ ]:
llm_with_structure = llm.with_structured_output(PydanticUserInfo)

In [ ]:
structured_output = llm_with_structure.invoke("The sky is blue")

In [ ]:
structured_output

In [ ]:
print(structured_output.name)

In [ ]:
print(structured_output.country)

In [ ]:
structured_output = llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)

In [ ]:
structured_output

## Dealing with Errors

In [ ]:
class Performer(BaseModel):
    """Filmography info about an actor/actress"""
    name: Annotated[str, Field(description="name of an actor/actress")]
    film_names: Annotated[List[str], Field(description="list of names of films they starred in")]

In [ ]:
llm_with_structure = llm.with_structured_output(Performer)

In [ ]:
response = llm_with_structure.invoke(
    "Generate the filmography for Scarlett Johansson. Top 5 only"
)
response

**Fixing Parser**

In [ ]:
response.json()

In [ ]:
parser = PydanticOutputParser(pydantic_object=Performer)

In [ ]:
parser.parse(response.json())

In [ ]:
misformatted_result = "{'name': 'Scarlett Johansson', 'film_names': ['The Avengers']}"

In [ ]:
try:
    parser.parse(misformatted_result)
except OutputParserException as e:
    print(e)

In [ ]:
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
new_parser.parse(misformatted_result)